In [204]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [205]:
from surprise import Dataset, Reader
from surprise import SVD, SVDpp, BaselineOnly

from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [206]:
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.lmf import LogisticMatrixFactorization

In [207]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

In [208]:
random_state_value = 42

rating_column_name = 'rating'

**Загружаем датасет**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [209]:
data_dir = 'Data_Rabota'

In [210]:
train_df = pd.read_parquet(f"{data_dir}/Processed_dataset.parquet")
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,1,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,1,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,1,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,1,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...
12292583,367641,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,367641,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,367641,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,367641,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [211]:
test_df = pd.read_parquet(f"{data_dir}/test_public_mfti.parquet")
test_df  

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 228125, 225604, 235544, 119314]"
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 113482, 240245, 221376, 242642, 235561, 191512, 222762, 231446, 109269, 205578]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 240655, 240659, 237341, 176563, 255299, 245637, 109275, 111505, 212953, 217549, 207108, 200475, 211355, 211449, 108956, 111071, 253239, 109649, 221735, 222030, 222174, 231207, 231022, 234380, 106944, 203948, 175325, 193534, 197505, 234769, 238933, 225709, 184466, 242642, 203015]"
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 232128, 237590, 259570, 244126, 229677, 135447]"
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 240521, 179568, 184858, 100103]"


In [212]:
nonrel_act_list = ['show_vacancy',
                'preview_click_vacancy',
                'click_favorite',
                'preview_click_favorite']

rel_act_list = ['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

In [213]:
def get_top_n_vacancies(rating_act_list, pos_vacancies_start=0, pos_vacancies_end=100):
    show_act = train_df[train_df['event_type'].isin(rating_act_list)].groupby('vacancy_id_')['event_type'].count()
    top_n_with_event_quant = show_act.sort_values(ascending=False).iloc[pos_vacancies_start:pos_vacancies_end].to_frame().reset_index()
    return top_n_with_event_quant['vacancy_id_'].unique()

In [214]:
# ТОП-50 для рейтинга по просмотрам

vac_start = 0
vac_end = 50
rating_act = ['show_vacancy']

top50vac = get_top_n_vacancies(rating_act,
                               pos_vacancies_start=vac_start,
                               pos_vacancies_end=vac_end)
top50vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251], dtype=int64)

In [215]:
# ТОП-80 для рейтинга по просмотрам

vac_start = 0
vac_end = 80
rating_act = ['show_vacancy']

top80vac = get_top_n_vacancies(rating_act,
                               pos_vacancies_start=vac_start,
                               pos_vacancies_end=vac_end)
top80vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251, 164588, 154411, 155539, 120252, 230707, 127352,
       214513, 182084, 249571, 207156, 168935, 239021, 184440, 106944,
       187360, 206350, 117525, 111592, 150283, 253678, 112506, 117532,
       220718, 151616, 109079, 111837, 136266, 164246, 182439, 111890],
      dtype=int64)

In [216]:
# ТОП-200 для рейтинга по просмотрам

vac_start = 0
vac_end = 200
rating_act = ['show_vacancy']

top200vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top200vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251, 164588, 154411, 155539, 120252, 230707, 127352,
       214513, 182084, 249571, 207156, 168935, 239021, 184440, 106944,
       187360, 206350, 117525, 111592, 150283, 253678, 112506, 117532,
       220718, 151616, 109079, 111837, 136266, 164246, 182439, 111890,
       257631, 143721, 248852, 171332, 128183, 140917, 111941, 212141,
       102794, 212325, 229689, 258378, 106293, 176171, 129787, 181275,
       190928, 242560, 195702, 191512, 144113, 209568, 219681, 197930,
       239624, 128278, 230322, 221108, 171602, 154423, 205606, 209422,
      

In [217]:
# Топ вакансий с 10 до 95

vac_start = 10
vac_end = 95
rating_act = ['show_vacancy']

top95vac_from10 = get_top_n_vacancies(rating_act,
                                                      pos_vacancies_start=vac_start,
                                                      pos_vacancies_end=vac_end)
top95vac_from10

array([250327, 111867, 110421, 108242, 242642, 217683, 258441, 158242,
       162187, 174953, 176141, 207108, 247535, 110792, 237341, 113305,
       240744, 149024, 244077, 113482, 180382, 114583, 182100, 113707,
       105907, 210628, 227708, 110793, 247276, 181745, 164481, 193331,
       246509, 243868, 190030, 153245, 138634, 169194, 115924, 126251,
       164588, 154411, 155539, 120252, 230707, 127352, 214513, 182084,
       249571, 207156, 168935, 239021, 184440, 106944, 187360, 206350,
       117525, 111592, 150283, 253678, 112506, 117532, 220718, 151616,
       109079, 111837, 136266, 164246, 182439, 111890, 257631, 143721,
       248852, 171332, 128183, 140917, 111941, 212141, 102794, 212325,
       229689, 258378, 106293, 176171, 129787], dtype=int64)

In [218]:
# Создаем колонку с типом действия для рейтинга
# 1 - целевое действие, 0 - остальные действия

def check_is_purpose(event):
    if event in rel_act_list:
        return 1
    else:
        return 0

train_df[rating_column_name] = train_df['event_type'].apply(check_is_purpose)
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,rating
0,1,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
2,1,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
3,1,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
4,1,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
...,...,...,...,...,...,...,...,...,...
12292583,367641,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292584,367641,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts,1
12292585,367641,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292586,367641,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy,0


In [219]:
# Готовим датафрейм для обучения модели:

train_matrix = train_df.groupby(['cookie_id', 'vacancy_id_'], as_index=False)[rating_column_name].sum()
train_matrix

,cookie_id,vacancy_id_,rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0
...,...,...,...
4678580,ffffdb17f45b4032b386d691d52e6c00,107125,1
4678581,ffffdb17f45b4032b386d691d52e6c00,107893,1
4678582,ffffdb17f45b4032b386d691d52e6c00,150843,1
4678583,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [220]:
# Функция для создания рейтинга на основе порогового значения для целевых действий

def create_rating(input_rating, threshold = 1):
    rating = input_rating
    if rating > threshold:
        rating = 1
    return rating

In [221]:
# Создаем рейтинг
# Принципы построения
# 1. rating >= 1 - rating = 1
# 2. rating = 0 - оставляем без изменения

rating_threshold = 1

train_matrix_0_1 = train_matrix.copy()
train_matrix_0_1[rating_column_name] = train_matrix_0_1[rating_column_name].apply(create_rating, threshold = rating_threshold)
train_matrix_0_1
train_matrix_0_1[rating_column_name].unique()

array([0, 1], dtype=int64)

In [222]:
# Создаем рейтинг
# Принципы построения
# 1. rating >= 2 (2 и больше целевых действия у пользователя) - rating = 2
# 2. rating = 1 и rating = 0 - оставляем без изменения

rating_threshold = 2

train_matrix_0_2 = train_matrix.copy()
train_matrix_0_2[rating_column_name] = train_matrix_0_2[rating_column_name].apply(create_rating, threshold = rating_threshold)
train_matrix_0_2
train_matrix_0_2[rating_column_name].unique()

array([0, 1, 2], dtype=int64)

In [223]:
# Готовим данные для обучения модели

reader = Reader(rating_scale=(0, 2))
train_data_0_1 = Dataset.load_from_df(train_matrix_0_1[['cookie_id', 'vacancy_id_', rating_column_name]], reader)

reader = Reader(rating_scale=(0, 2))
train_data_0_2 = Dataset.load_from_df(train_matrix_0_2[['cookie_id', 'vacancy_id_', rating_column_name]], reader)

In [224]:
# Обучаем модель для датасета с рейтингом 0 или 1

svd_20epochs_0_1 = SVD(verbose=True, n_epochs=20)
trainset_0_1 = train_data_0_1.build_full_trainset()
svd_20epochs_0_1.fit(trainset_0_1)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [225]:
# Обучаем модель для датасета с рейтингом 0, 1 или 2

svd_20epochs_0_2 = SVD(verbose=True, n_epochs=20)
trainset_0_2 = train_data_0_2.build_full_trainset()
svd_20epochs_0_2.fit(trainset_0_2)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [226]:
col_names ={
    'id_col_name': 'cookie_id',
    'vac_col_name': 'vacancy_id_',
    'rating_col_name': rating_column_name
}

In [260]:
def transform_into_sparse_matrix(input_df, col_names):
    input_df_fixed = input_df.copy()
    input_df_fixed[col_names['id_col_name']] = input_df_fixed[col_names['id_col_name']].astype('category')
    input_df_fixed[col_names['vac_col_name']] = input_df_fixed[col_names['vac_col_name']].astype('category')
    input_df_fixed[col_names['rating_col_name']] = input_df_fixed[col_names['rating_col_name']].astype('uint8')
    user_item_matrix = csr_matrix((input_df_fixed[col_names['rating_col_name']], 
                                (input_df_fixed[col_names['id_col_name']].cat.codes, 
                                 input_df_fixed[col_names['vac_col_name']].cat.codes)))
    id_cat_codes = dict(zip(input_df_fixed[col_names['id_col_name']], input_df_fixed[col_names['id_col_name']].cat.codes))
    vac_cat_codes = dict(zip(input_df_fixed[col_names['vac_col_name']], input_df_fixed[col_names['vac_col_name']].cat.codes))
    cat_vac_codes = dict(zip(input_df_fixed[col_names['vac_col_name']].cat.codes, input_df_fixed[col_names['vac_col_name']]))
    return (user_item_matrix, id_cat_codes, vac_cat_codes, cat_vac_codes)


In [267]:
Impl_0_1_set = transform_into_sparse_matrix(train_matrix_0_1, col_names)

{'0000c4548c3944c08972bbdc1fa4eb85': 0,
 '0000d7508334414ca792c5ff66eb8c14': 1,
 '0000e63f423542fe8090575b611cea4d': 2,
 '00013bf3e58244b48d4dbdf43543c30c': 3,
 '00015019b0594984afc3fd9fa9557490': 4,
 '0001a23eb7fa4b1db1e933484b03e668': 5,
 '00021ea9459044b0af294eb5dd7ab8a7': 6,
 '00025878f0534a6a96dac2042dfa07b7': 7,
 '00028e0b0d474748ac5a83688a4f8896': 8,
 '0002cb657088464b99775a69ce83a2ef': 9,
 '000344ddc0664127b54f529eaf183e30': 10,
 '000385da780542d5a2437cc416e4f063': 11,
 '00038b5200304d5d96a05a31ca94751a': 12,
 '0003e3a2e8c84a4ebfd3a58b7327affc': 13,
 '0003e60e0c684618baf363fcc19bc2a4': 14,
 '00040a7c359b436eada01587aab01945': 15,
 '0004451a95a04432b6d8dce117bf3c0c': 16,
 '00044a0333bc4f44bb80ccf9fc678dba': 17,
 '000478d4b2964d1b8e53ec9c0e050df3': 18,
 '0004b0d059984db291d418066b9f1502': 19,
 '0004c1c453a348cd993ef076a8fcafe4': 20,
 '000537e8c0224eb78b04dd3827b1be91': 21,
 '00053e2a426f444f9a71220b392ddd4a': 22,
 '00055873d53a4eaeb16d615d263497f1': 23,
 '00057860633f4969b8a28a43

In [229]:
Impl_ALS_0_1 = AlternatingLeastSquares(factors=100,
                                       regularization=0.05,
                                       calculate_training_loss=True,
                                       random_state=random_state_value) # , alpha=3.0
Impl_ALS_0_1.fit(Impl_0_1_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [230]:
Impl_LMF_0_1 = LogisticMatrixFactorization(factors=50, learning_rate=2, random_state=42)
Impl_LMF_0_1.fit(user_item_0_1[0])

  0%|          | 0/30 [00:00<?, ?it/s]

In [262]:
Impl_0_2_set = transform_into_sparse_matrix(train_matrix_0_2, col_names)

In [232]:
Impl_ALS_0_2 = AlternatingLeastSquares(factors=100,
                                       regularization=0.05,
                                       calculate_training_loss=True,
                                       random_state=random_state_value) # , alpha=3.0
Impl_ALS_0_2.fit(Impl_0_2_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [233]:
Impl_LMF_0_2 = LogisticMatrixFactorization(factors=50, learning_rate=2, random_state=42)
Impl_LMF_0_2.fit(Impl_0_2_set[0])

  0%|          | 0/30 [00:00<?, ?it/s]

In [297]:
# Вспомогательные функции

def get_predictions(cookie_id, model_info, initial_set, n_predict = 5):
    # Получим все вакансии с которыми взаимодействовал cookie_id:
    interacted_items = train_matrix[train_matrix['cookie_id'] == cookie_id]['vacancy_id_'].unique()
    
    # Удалим из этого перечня вакансии с которыми контактировал юзер:
    items_to_predict = np.setdiff1d(initial_set, interacted_items)
    
    # Создадим тест датасет для данного юзера:
    test_set = [[cookie_id, item, 4.] for item in items_to_predict]    
    
    model = model_info[0]
    model_type = model_info[1]  
    
    # получим топ-n вакансий для данного юзера:
    if model_type == 'surprise':
        top_n_vacancies = get_predictions_surprise(cookie_id, model, test_set, n_predict)
    
    if model_type == 'implicit':
        top_n_vacancies = get_predictions_implicit(cookie_id, model, items_to_predict, model_info[2], n_predict)
    
    return top_n_vacancies

def get_predictions_surprise(cookie_id, model, test_set, n_predict = 5):
    predictions = model.test(test_set)
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_predict]
    return [pred.iid for pred in top_n]

def get_predictions_implicit(cookie_id, model, test_set, impl_set, n_predict = 5):
    user_item = impl_set[0]
    id_cat_code = impl_set[1][cookie_id]
    vac_cat_codes = impl_set[2]
    cat_vac_codes = impl_set[3]
    filter_items = np.array([vac_cat_codes[i] for i in test_set])
    ids, scores = model.recommend(id_cat_code,
                                  user_item[id_cat_code],
                                  N=n_predict,
                                  filter_items=filter_items,
                                  filter_already_liked_items=True)
    return [cat_vac_codes[i] for i in ids]    

def calc_correct_recomend_number(predict_col, vac_col):
    """Функция для определения количества совпадающих вакансий в двух списках"""
    return len(set(predict_col) & set(vac_col))


def get_prec_n(res_df, num_corr_column, n_predict = 5):
    """Функция для расчета precision@n метрики"""
    corr_recommend = res_df[num_corr_column].sum()
    prec_5 = corr_recommend / (n_predict * res_df.shape[0])
    return {'num_corr_recomend': corr_recommend, f'precision@{n_predict}': prec_5}

In [235]:
# Копируем датасет чтобы не менять изначальный

test_df_with_pred = test_df.copy()

In [305]:
# Задаем словари с моделями и датасетами чтобы перебрать их все

models_dict = {
    'SVD_20ep_rat01': [svd_20epochs_0_1, 'surprise'],
    'SVD_20ep_rat012': [svd_20epochs_0_2, 'surprise'],
    'Impl_ALS_rat01': [Impl_ALS_0_1, 'implicit', Impl_0_1_set],
    'Impl_ALS_rat012': [Impl_ALS_0_2, 'implicit', Impl_0_2_set]
}

set_dict = {
    'top50': top50vac,
    'top80': top80vac,
    'top200': top200vac,
    'top95_10': top95vac_from10
}

In [306]:
# Импортируем tqdm для progress bar
from tqdm.auto import tqdm

def get_res_df(models_dict, set_dict, n_predict = 5):
    res_dict = {}
    for model_name in tqdm(models_dict):        
        for set_name in tqdm(set_dict):        
            # Создаем столбец с предсказаниями
            predict_col_name = f'predict_id_{model_name}_{set_name}_n_{n_pred}'            
            test_df_with_pred[predict_col_name] = test_df_with_pred['cookie_id'].apply(get_predictions,
                                                                                       model_info = models_dict[model_name],
                                                                                       initial_set = set_dict[set_name],
                                                                                       n_predict = n_pred)                  
            
            # Создаем столбец с количеством правильных предсказаний для каждого id
            num_corr_col_name = f'num_corr_{model_name}_{set_name}_n_{n_pred}'        
            test_df_with_pred[num_corr_col_name] = test_df_with_pred.apply(
                lambda x: calc_correct_recomend_number(x[predict_col_name], x['vacancy_id_']), axis=1
            )
            
            # Считаем precision@n метрику
            res_dict[model_name+'_'+set_name] = get_prec_n(test_df_with_pred,
                                                           num_corr_column = num_corr_col_name,
                                                           n_predict = n_pred)
    return pd.DataFrame.from_dict(res_dict, orient='index')

In [307]:
n_pred = 5

res_df = get_res_df(models_dict, set_dict, n_predict = n_pred)
res_df

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

,num_corr_recomend,precision@5
SVD_20ep_rat01_top50,43,0.011140
SVD_20ep_rat01_top80,81,0.020984
SVD_20ep_rat01_top200,39,0.010104
SVD_20ep_rat01_top95_10,114,0.029534
SVD_20ep_rat012_top50,55,0.014249
SVD_20ep_rat012_top80,85,0.022021
SVD_20ep_rat012_top200,37,0.009585
SVD_20ep_rat012_top95_10,119,0.030829
Impl_ALS_rat01_top50,88,0.022798
Impl_ALS_rat01_top80,80,0.020725
